# Draw Heatmap

In [1]:
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random
import torch

## MODIFY THE BLOCK BELOW

Modify the result folder name.

In [ ]:
with open("./results/example_addstrict_exp/records.pkl", "rb") as f:
    records = pickle.load(f)
df_records = pd.DataFrame(records)
df_records.head(3)

In [ ]:
df_records["output_similarity_cos"].mean()

In [4]:
def compute_output_exp_similarity(self, traj1, traj2, gamma=1.0):
    vec1 = traj1.flatten()
    vec2 = traj2.flatten()
    squared_diff = np.sum((vec1 - vec2) ** 2)
    similarity = np.exp(-gamma * squared_diff)
    return similarity

## MODIFY THE CODE BELOW

Modify the result folder name.

In [ ]:
with open("./results/example_addstrict_exp/deletion_records.pkl", "rb") as f:
    deletion_records = pickle.load(f)
df_deletion_records = pd.DataFrame(deletion_records)
list_deleted_tokens = df_deletion_records["token"].tolist()
df_deletion_records.head(3)

In [6]:
deleted_demo_input = []
deleted_demo_output = []
general_demo_input = []
general_demo_output = []

for record in records:
    if record["memory_token"] in list_deleted_tokens:
        deleted_demo_input.append(record["input_similarity"])
        deleted_demo_output.append(record["output_similarity_exp"])
    else:
        general_demo_input.append(record["input_similarity"])
        general_demo_output.append(record["output_similarity_exp"])

datasets = [
    (deleted_demo_input, deleted_demo_output),
    (general_demo_input, general_demo_output),
]

In [ ]:
plt.style.use("seaborn-whitegrid")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
axes = [ax1, ax2]
titles = ["Deleted Records", "Retained Records"]


for ax, title, (x_data, y_data) in zip(axes, titles, datasets):
    # Create 2D histogram with more bins for smoother visualization
    heatmap, xedges, yedges = np.histogram2d(
        x_data, y_data, bins=30, range=[[0.5, 1], [0.5, 1]]
    )
    # Normalize and smooth the heatmap
    heatmap = heatmap.T / np.max(heatmap)

    # Create heatmap plot with improved colormap
    im = ax.imshow(
        heatmap,
        origin="lower",
        extent=[0.5, 1, 0.5, 1],
        aspect="auto",
        cmap="viridis",
    )

    # Customize subplot with professional styling
    # ax.set_title(title, pad=15, fontsize=20)
    ax.set_title(title, fontsize=34, pad=5, fontweight='bold')  # Reduced pad value to bring title closer
    # ax.set_title(title, fontsize=20)
    ax.set_xlabel("Input Similarity", fontsize=34, fontweight='bold')
    if ax == ax1:
        ax.set_ylabel("Output Similarity", fontsize=34, fontweight='bold')

    # Customize ticks
    ax.tick_params(axis="both", which="major", labelsize=16)

    # Add subtle grid
    ax.grid(True, linestyle="--", alpha=0.3, color="gray")

    # Add colorbar with custom styling
    cbar = plt.colorbar(im, ax=ax)
    cbar.ax.tick_params(labelsize=14)
    # cbar.set_label('Normalized Density', fontsize=14, labelpad=10)

    # Add subtle spines
    for spine in ax.spines.values():
        spine.set_color("lightgray")
        spine.set_linewidth(0.5)

# Add main title with professional styling
plt.suptitle(
    "Input vs Output Similarity Analysis for AgentDriver",
    fontsize=34,
    fontweight="bold",
)

# Adjust layout to prevent overlapping
plt.tight_layout()

# Save with high quality
plt.savefig("./plots/updated_similarity_heatmaps.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()